# Demand Planning Tool - Production Forecast v3

This notebook creates a comprehensive demand planning analysis with:
- Historical sales analysis by Product Name/SKU and channel
- **Improved statistical forecasts** using 2-year product-type seasonality + weighted trend
- Seasonal indices calculated at **product type level** (pooled across all SKUs in a type)
- Recency-weighted trend per SKU (recent months weighted more heavily)
- Forecast pivot tabs show last 2 years of actuals alongside forecast for direct comparison
- Export to Google Sheets

## Forecasting Methodology (v3)
- **Seasonality**: Pooled across all SKUs within a product type, using the last 24 months of TOTAL channel data. Seasonal indices are stable because they draw on the full volume of a product type rather than a single item.
- **Trend**: Linear trend via weighted least-squares on each SKU's last 12 months (deseasonalized). Recent months weighted up to 12× heavier.
- **Base Level**: Exponentially weighted moving average of last 12 months per SKU.
- **Manual Growth Override**: Optional annual growth rate applied on top of data-driven trend.

## Setup Instructions
1. Upload your CSV file when prompted
2. Run all cells in order
3. Authenticate with Google when prompted
4. The output will be saved to your Google Drive

In [1]:
# Install required packages
!pip install gspread oauth2client pandas numpy openpyxl scipy -q

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from google.colab import files
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials

In [3]:
# Upload your CSV file
print("Please upload your sales data CSV file:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"\nFile '{filename}' uploaded successfully!")

Please upload your sales data CSV file:


Saving _SELECT_mts_created_date_mts_products__master__id_mts_products___202602171733.csv to _SELECT_mts_created_date_mts_products__master__id_mts_products___202602171733.csv

File '_SELECT_mts_created_date_mts_products__master__id_mts_products___202602171733.csv' uploaded successfully!


In [4]:
# ============================================================
# STEP 2: LOAD SALES DATA
# ============================================================
# Upload your sales CSV file or specify a pre-uploaded filename
# ============================================================

# Option 1: Upload a new sales CSV file (uncomment to use)
# uploaded = files.upload()
# if uploaded:
#     filename = list(uploaded.keys())[0]

# Option 2: Use a pre-uploaded file (default)
filename = '_SELECT_mts_created_date_mts_products__master__id_mts_products___202602171733.csv'  # Change this to your uploaded filename

print(f"Loading data from: {filename}")

try:
    df = pd.read_csv(filename)

    print(f"✅ Data loaded successfully")
    print(f"   Total rows: {len(df):,}")
    print(f"   Date range: {df['created_date'].min()} to {df['created_date'].max()}")
    print(f"   Channels: {df['orders__source'].unique()}")
    print(f"   Unique SKUs: {df['products__variants__sku'].nunique()}")
    print(f"\nFirst few rows:")
    display(df.head())

except FileNotFoundError:
    print(f"⚠️  File not found: {filename}")
    print("   Please upload the file first, or uncomment the upload section above.")
    raise
except Exception as e:
    print(f"⚠️  Error loading file: {e}")
    raise


Loading data from: _SELECT_mts_created_date_mts_products__master__id_mts_products___202602171733.csv
✅ Data loaded successfully
   Total rows: 84,579
   Date range: 2022-01-01 to 2026-02-16
   Channels: ['Direct-to-Consumer' 'Wholesale' 'Kristina Holey']
   Unique SKUs: 275

First few rows:


,created_date,products__master__id,products__variants__sku,products__variants__title,products__root_product__title,products__product_type,products__vendor,orders__source,quantity,price,total_gross_sales,total_net_sales,total_sales
0,2022-01-01,19215247238,FG-10047,Vitamins C + E + Ferulic Serum - Retail (1 oz),Vitamins C + E + Ferulic Serum,SERUM,Marie Veronique,Direct-to-Consumer,7,90.0,630.0,630.0,630.0
1,2022-01-01,19215265286,FG-10029,Protective Day Oil - Retail (1 oz),Protective Day Oil,OIL,Marie Veronique,Direct-to-Consumer,7,65.0,455.0,447.2,447.2
2,2022-01-01,29370167116,FG-10006,Barrier Restore Serum - Retail (1 oz),Barrier Restore Serum,SERUM,Marie Veronique,Direct-to-Consumer,4,110.0,440.0,440.0,440.0
3,2022-01-01,32378481934372,FG-10005,Barrier Lipid Complex - Retail (1 oz),Barrier Lipid Complex,OIL,Marie Veronique,Direct-to-Consumer,4,95.0,380.0,380.0,380.0
4,2022-01-01,29370177804,FG-10038,Soothing B3 Serum - Retail (1 oz),Soothing B3 Serum,SERUM,Marie Veronique,Direct-to-Consumer,3,90.0,270.0,270.0,270.0


In [5]:
# Prepare data for analysis
df['created_date'] = pd.to_datetime(df['created_date'])
df['year_month'] = df['created_date'].dt.to_period('M')
df['year'] = df['created_date'].dt.year
df['month'] = df['created_date'].dt.month

# Aggregate to monthly level by SKU and channel
monthly_data = df.groupby(['year_month', 'products__variants__sku', 'orders__source'])['quantity'].sum().reset_index()
monthly_data['year_month_str'] = monthly_data['year_month'].astype(str)

# Get SKU details
sku_details = df.groupby('products__variants__sku').agg({
    'products__variants__title': 'first',
    'products__root_product__title': 'first',
    'products__product_type': 'first'
}).reset_index()

# Standardize product_name: prefer root product title, fall back to variant title
sku_details['product_name'] = sku_details['products__root_product__title'].fillna(
    sku_details['products__variants__title']
)

print("Data aggregated to monthly level")
print(f"Monthly records: {len(monthly_data):,}")
print(f"\nSample SKU → Product Name mapping:")
print(sku_details[['products__variants__sku', 'product_name', 'products__product_type']].head(10).to_string(index=False))

Data aggregated to monthly level
Monthly records: 7,654

Sample SKU → Product Name mapping:
products__variants__sku                            product_name products__product_type
        $20 Reward Code                         $20 Reward Code                   None
     000000000300088687 Quinton Hypertonic Ampoules 30 Servings                   None
           500-V0-40-DR                      Protective Day Oil                    OIL
           BLANKET-V4-1       HigherDose Blanket with No Insert                   WRAP
      BLANKET-W-1INSERT      HigherDose Blanket with One Insert                   WRAP
      BLANKET-W-3INSERT    Infrared Sauna Blanket by HigherDOSE                   WRAP
      BOTTLES+ CLOSURES                       BOTTLES+ CLOSURES                   None
                 CC3302                     The Cleansing Coins                   None
  DHL EXPRESS WORLDWIDE                   DHL EXPRESS WORLDWIDE                   None
              FG-100004               

In [6]:
# ============================================================
# STEP 1: BUILD PRODUCT-TYPE SEASONAL INDICES
# ============================================================
# Seasonality is pooled across ALL SKUs within each product type,
# using the last 24 months of TOTAL channel data.
# This gives stable, noise-resistant seasonal patterns.
# ============================================================

MONTH_NAMES = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
    5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
    9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}

def build_product_type_seasonal_indices(monthly_data, sku_details, cutoff='2026-01'):
    """
    Calculate seasonal indices per product type per calendar month.
    Uses TOTAL channel (all channels combined) for the last 24 months up to cutoff.
    Returns a dict: {product_type: {month_number: index}}
    """
    # Work from TOTAL channel (sum all channels per SKU-month)
    total_monthly = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()

    # Attach product type
    total_monthly = total_monthly.merge(
        sku_details[['products__variants__sku', 'products__product_type']],
        on='products__variants__sku', how='left'
    )

    # Filter to last 24 months up to cutoff
    cutoff_period = pd.Period(cutoff, freq='M')
    start_period = cutoff_period - 23  # 24 months window
    total_monthly = total_monthly[
        (total_monthly['year_month_str'] >= str(start_period)) &
        (total_monthly['year_month_str'] <= cutoff)
    ].copy()

    total_monthly['cal_month'] = total_monthly['year_month_str'].str[5:7].astype(int)

    # Sum quantity by product_type + calendar month (across all SKUs and both years)
    pt_monthly = total_monthly.groupby(['products__product_type', 'cal_month'])['quantity'].sum().reset_index()

    # Calculate average monthly quantity per product type per calendar month
    # (divide by number of years represented — up to 2)
    # Then normalize so indices average to 1.0 across the 12 months
    seasonal_indices = {}
    product_types = pt_monthly['products__product_type'].unique()

    for pt in product_types:
        pt_data = pt_monthly[pt_monthly['products__product_type'] == pt].copy()

        # Build month → avg quantity mapping
        month_qty = {}
        for _, row in pt_data.iterrows():
            month_qty[int(row['cal_month'])] = row['quantity']

        # Fill missing months with average of available months
        if len(month_qty) > 0:
            avg_qty = np.mean(list(month_qty.values()))
        else:
            avg_qty = 1.0

        for m in range(1, 13):
            if m not in month_qty:
                month_qty[m] = avg_qty

        # Normalize: each index = month_qty / (sum of all months / 12)
        grand_avg = np.mean([month_qty[m] for m in range(1, 13)])
        if grand_avg > 0:
            indices = {m: month_qty[m] / grand_avg for m in range(1, 13)}
        else:
            indices = {m: 1.0 for m in range(1, 13)}

        seasonal_indices[pt] = indices

    return seasonal_indices


# Build product-type seasonal indices
pt_seasonal_indices = build_product_type_seasonal_indices(monthly_data, sku_details)

print(f"✅ Seasonal indices built for {len(pt_seasonal_indices)} product types")
print("\nProduct types found:")
for pt in sorted(pt_seasonal_indices.keys()):
    print(f"  • {pt}")

✅ Seasonal indices built for 25 product types

Product types found:
  • BACKBAR
  • BODY
  • BOOK
  • BUNDLE
  • CLEANSER
  • CONDITIONER
  • DUO
  • FREEGIFT_HIDDEN
  • GUA SHA
  • KIT
  • MASK
  • MIST
  • OIL
  • PACKAGING
  • PROTECT
  • SAMPLE
  • SERUM
  • SET
  • SHAMPOO
  • SOAP & LOTION DISPENSERS
  • SUNSCREEN
  • SUPPLEMENT
  • TINCTURE
  • VIRTUAL CONSULTATION
  • WRAP


In [7]:
# ============================================================
# STEP 2: DISPLAY SEASONALITY TABLE (Product Type × Month)
# ============================================================
# This shows the seasonal index for each product type by month.
# Index > 1.0 = that month is stronger than average
# Index < 1.0 = that month is weaker than average
# ============================================================

seasonality_rows = []
for pt, indices in sorted(pt_seasonal_indices.items()):
    row = {'Product Type': pt}
    for m in range(1, 13):
        row[MONTH_NAMES[m]] = round(indices[m], 3)
    seasonality_rows.append(row)

seasonality_display_df = pd.DataFrame(seasonality_rows)

print("📊 SEASONAL INDICES BY PRODUCT TYPE AND MONTH")
print("   (Based on last 24 months of TOTAL channel data)")
print("   Index > 1.0 = stronger than annual average | Index < 1.0 = weaker than annual average")
print()
print(seasonality_display_df.to_string(index=False))
print()
print("Note: Indices within each product type sum to 12.0 (average = 1.0)")

📊 SEASONAL INDICES BY PRODUCT TYPE AND MONTH
   (Based on last 24 months of TOTAL channel data)
   Index > 1.0 = stronger than annual average | Index < 1.0 = weaker than annual average

            Product Type   Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct   Nov   Dec
                 BACKBAR 0.842 1.000 1.000 1.000 1.000 1.474 1.684 0.947 1.158 0.526 0.737 0.632
                    BODY 0.837 0.378 0.610 0.717 0.511 0.589 0.567 1.083 0.646 2.992 1.577 1.493
                    BOOK 0.589 0.547 1.221 0.926 0.968 1.221 1.011 0.547 0.758 0.800 1.474 1.937
                  BUNDLE 1.585 0.740 0.820 0.563 0.353 0.282 0.311 0.458 0.664 0.370 4.898 0.954
                CLEANSER 0.959 1.051 1.037 0.790 0.881 0.879 1.027 0.921 1.214 0.806 1.561 0.877
             CONDITIONER 1.000 1.000 1.103 0.828 1.655 0.276 0.828 1.103 1.379 0.828 1.000 1.000
                     DUO 1.000 0.996 1.371 0.972 1.021 1.156 0.947 1.175 1.027 0.935 0.400 1.000
         FREEGIFT_HIDDEN 0.008 0.280 0

In [8]:
# ============================================================
# STEP 3: FORECASTING ENGINE (using product-type seasonality)
# ============================================================

def calculate_weighted_trend(deseasonalized_values):
    """
    Estimate trend (units/month) using weighted least-squares.
    Weights increase linearly so most recent = window weight, oldest = 1.
    """
    n = len(deseasonalized_values)
    if n < 4:
        return 0.0

    weights = np.arange(1, n + 1, dtype=float)
    x = np.arange(n, dtype=float)
    w = weights
    wx = (w * x).sum()
    wy = (w * deseasonalized_values).sum()
    wxx = (w * x * x).sum()
    wxy = (w * x * deseasonalized_values).sum()
    wsum = w.sum()

    denom = wsum * wxx - wx * wx
    if abs(denom) < 1e-10:
        return 0.0

    slope = (wsum * wxy - wx * wy) / denom
    return slope


def exponential_weighted_mean(arr, alpha=0.15):
    """
    Exponentially weighted mean — most recent observation has highest weight.
    """
    n = len(arr)
    weights = np.array([(1 - alpha) ** (n - 1 - i) for i in range(n)])
    return np.dot(weights, arr) / weights.sum()


def calculate_forecast(historical_values, calendar_months, product_type,
                       pt_seasonal_indices, forecast_calendar_months,
                       growth_rate=0.0):
    """
    Forecast using PRODUCT TYPE seasonal indices.

    Parameters:
    - historical_values       : numpy array of quantities (sorted oldest→newest)
    - calendar_months         : numpy array of calendar month numbers (1-12) for history
    - product_type            : string product type for seasonal index lookup
    - pt_seasonal_indices     : dict {product_type: {month: index}}
    - forecast_calendar_months: list of calendar month numbers (1-12) for forecast
    - growth_rate             : manual annual growth override
    """
    n = len(historical_values)
    forecast_periods = len(forecast_calendar_months)

    # Get seasonal indices for this product type (fallback to flat 1.0)
    seasonal_indices = pt_seasonal_indices.get(product_type, {m: 1.0 for m in range(1, 13)})

    if n < 4:
        avg = np.mean(historical_values) if n > 0 else 0
        forecasts = []
        for cal_m in forecast_calendar_months:
            forecasts.append(int(round(max(0, avg * seasonal_indices.get(cal_m, 1.0)))))
        return forecasts

    # Deseasonalize last 12 months of history
    window = min(n, 12)
    recent_vals = historical_values[-window:]
    recent_months = calendar_months[-window:]

    deseason_recent = np.array([
        v / seasonal_indices.get(int(m), 1.0) if seasonal_indices.get(int(m), 1.0) > 0 else v
        for v, m in zip(recent_vals, recent_months)
    ])

    # Base level: exponentially weighted mean of deseasonalized recent
    base_level = exponential_weighted_mean(deseason_recent, alpha=0.15)

    # Trend: weighted least squares on deseasonalized recent
    trend_slope = calculate_weighted_trend(deseason_recent)

    # Cap trend at ±2.5% of base per month
    max_slope = base_level * 0.025 if base_level > 0 else 1.0
    trend_slope = np.clip(trend_slope, -max_slope, max_slope)

    # Manual growth → monthly multiplier
    monthly_growth = (1 + growth_rate) ** (1 / 12) - 1

    forecasts = []
    for i, cal_month in enumerate(forecast_calendar_months):
        projected_base = base_level + trend_slope * (i + 1)
        growth_factor = (1 + monthly_growth) ** (i + 1)
        projected_base *= growth_factor
        seasonal_factor = seasonal_indices.get(int(cal_month), 1.0)
        forecast = projected_base * seasonal_factor
        forecasts.append(int(round(max(0, forecast))))

    return forecasts


print("✅ Forecasting engine loaded (product-type seasonality)")

✅ Forecasting engine loaded (product-type seasonality)


In [9]:
# ============================================================================
# TUNE YOUR FORECAST GROWTH RATES HERE — per channel
# ============================================================================
# Set an annual growth rate for each channel independently.
# This is a MANUAL OVERRIDE on top of the data-driven statistical trend.
# Leave at 0.0 to rely purely on the model's trend for that channel.
#
# Examples:
#   0.10  = add 10% annual growth on top of data trend
#   0.05  = add 5% growth
#   0.0   = pure statistical forecast (recommended starting point)
#  -0.10  = force 10% decline on top of data trend
#  -0.20  = force 20% decline (use to rein in an over-optimistic channel)
#
# TOTAL is computed independently from DTC + Wholesale, so set it
# separately if you want the TOTAL tab to reflect a blended view.
# ============================================================================

CHANNEL_GROWTH_RATES = {
    'Direct-to-Consumer': 0.0,   # ← tune DTC here
    'Wholesale':          0.0,   # ← tune Wholesale here
    'TOTAL':              0.0,   # ← tune Total here
}

print("Per-Channel Growth Rate Overrides:")
for ch, rate in CHANNEL_GROWTH_RATES.items():
    print(f"  {ch:<25} {rate*100:+.1f}% annually")


Per-Channel Growth Rate Overrides:
  Direct-to-Consumer        +0.0% annually
  Wholesale                 +0.0% annually
  TOTAL                     +0.0% annually


In [10]:
# Active SKU List (hardcoded)
ACTIVE_SKUS = [
    'FG-10082', 'FG-90004', 'FG-10004', 'FG-80004', 'FG-20004',
    'FG-90005', 'FG-10005', 'FG-90006', 'FG-10006', 'FG-90067',
    'FG-10081', 'FG-20081', 'FG-90007', 'FG-10007', 'FG-90103',
    'FG-10103', 'FG-90008', 'FG-10083', 'FG-10101', 'FG-90012',
    'FG-10014XL', 'FG-10014L', 'FG-10014M', 'FG-10094', 'FG-10022',
    'FG-90017', 'FG-10017', 'FG-10018', 'FG-10019', 'FG-90022',
    'FG-10025', 'FG-10026', 'FG-20026', 'FG-90028', 'FG-10028',
    'FG-90029', 'FG-10029', 'FG-90030', 'FG-10030', 'FG-20030',
    'FG-90100', 'FG-10100', 'FG-90034', 'FG-10034', 'FG-90036',
    'FG-10036', 'FG-90099', 'FG-10099', 'FG-10068', 'FG-90038',
    'FG-10038', 'FG-90102', 'FG-10102', 'FG-90104', 'FG-10104',
    'FG-90047', 'FG-10047'
]

active_skus_from_file = set(ACTIVE_SKUS)
print(f"Active SKUs loaded (hardcoded): {len(active_skus_from_file)}")
print(f"Sample: {list(active_skus_from_file)[:5]}")


Active SKUs loaded (hardcoded): 57
Sample: ['FG-90067', 'FG-10014XL', 'FG-90104', 'FG-10104', 'FG-10014M']


In [11]:
# ============================================================
# STEP 4: GENERATE FORECASTS
# ============================================================
# ACTIVE SKU FILTER: Reads from a CSV file of active SKUs.
# You can either upload a new file or use the default filename.
# Expected format: Single column with SKU codes (header optional).
# ============================================================

forecast_months = pd.period_range('2026-02', '2026-12', freq='M')
forecast_cal_months = [p.month for p in forecast_months]

all_channels = ['Direct-to-Consumer', 'Wholesale', 'TOTAL']

# ── Active SKU filter already set by previous cell ──────────────────────
# active_skus_from_file is defined above with the hardcoded list

print(f"\nActive SKU filter: {len(active_skus_from_file)} SKUs")

# Debug: show what SKUs are in data vs filter
all_data_skus = set(sku_details['products__variants__sku'].unique())
matching = active_skus_from_file & all_data_skus
print(f"Matching SKUs (in both filter and data): {len(matching)}")

if len(matching) == 0:
    print("⚠️  WARNING: NO MATCHING SKUs!")
    print("   Proceeding to forecast ALL SKUs.")
    active_skus_from_file = set()

# --- Generate forecasts (active SKUs only) ---
forecast_results = []
skipped_counts = {ch: 0 for ch in all_channels}
forecasted_skus = set()

for channel in all_channels:
    print(f"\nGenerating forecasts for {channel}...")

    if channel == 'TOTAL':
        channel_data = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()
    else:
        channel_data = monthly_data[monthly_data['orders__source'] == channel].copy()

    channel_skus = channel_data['products__variants__sku'].unique()

    for sku in channel_skus:
        sku_match = sku_details[sku_details['products__variants__sku'] == sku]
        if len(sku_match) == 0:
            continue
        sku_info = sku_match.iloc[0]
        product_type = sku_info['products__product_type']

        # ACTIVE SKU CHECK — only forecast if SKU is in uploaded file
        # If file is empty or failed to load, forecast all
        if len(active_skus_from_file) > 0 and sku not in active_skus_from_file:
            skipped_counts[channel] += 1
            continue

        sku_data = channel_data[channel_data['products__variants__sku'] == sku]

        # Historical data up to cutoff, sorted chronologically
        HISTORY_CUTOFF = '2026-01'
        historical = sku_data[sku_data['year_month_str'] <= HISTORY_CUTOFF].sort_values('year_month_str')
        hist_values = historical['quantity'].values.astype(float)
        hist_cal_months = np.array([int(ym[5:7]) for ym in historical['year_month_str'].values])

        # Forecast using product-type seasonality
        forecasts = calculate_forecast(
            hist_values,
            hist_cal_months,
            product_type,
            pt_seasonal_indices,
            forecast_cal_months,
            CHANNEL_GROWTH_RATES.get(channel, 0.0)
        )

        for month, forecast_qty in zip(forecast_months, forecasts):
            forecast_results.append({
                'channel': channel,
                'product_name': sku_info['product_name'],
                'sku': sku,
                'product_type': product_type,
                'month': str(month),
                'forecast_qty': forecast_qty
            })

        if channel == 'TOTAL':
            forecasted_skus.add(sku)

forecast_df = pd.DataFrame(forecast_results)
print(f"\n✅ Forecasts generated: {len(forecast_df):,} records")
print(f"   Unique SKUs forecasted (TOTAL channel): {len(forecasted_skus)}")
print(f"   SKUs skipped per channel: { {k: v for k, v in skipped_counts.items()} }")

# Final verification
if len(active_skus_from_file) > 0:
    print(f"\n📋 Filter was applied: {len(active_skus_from_file)} SKUs from file")
    print(f"   {len(forecasted_skus)} SKUs actually forecasted")
    if len(forecasted_skus) != len(matching):
        print(f"   ⚠️  Expected {len(matching)} forecasted SKUs (matching count)")
else:
    print(f"\n⚠️  No filter applied - all {len(forecasted_skus)} SKUs in data were forecasted")



Active SKU filter: 57 SKUs
Matching SKUs (in both filter and data): 57

Generating forecasts for Direct-to-Consumer...

Generating forecasts for Wholesale...

Generating forecasts for TOTAL...

✅ Forecasts generated: 1,628 records
   Unique SKUs forecasted (TOTAL channel): 57
   SKUs skipped per channel: {'Direct-to-Consumer': 183, 'Wholesale': 93, 'TOTAL': 218}

📋 Filter was applied: 57 SKUs from file
   57 SKUs actually forecasted


In [12]:
# ============================================================
# STEP 5: BUILD FORECAST COMPARISON PIVOT
# (2024 actuals | 2025 actuals | 2026 forecast — all months)
# ============================================================

def build_comparison_pivot(channel, monthly_data, forecast_df, sku_details):
    """
    Build a pivot table showing:
      Rows: Product Name | SKU
      Columns: 2024-01 ... 2024-12, 2025-01 ... 2025-12, 2026-01 (actual), 2026-02 ... 2026-12 (forecast)
    Adds a row-level label prefix so actuals vs forecast are clear.
    """
    # --- Historical actuals ---
    if channel == 'TOTAL':
        hist_data = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()
    else:
        hist_data = monthly_data[monthly_data['orders__source'] == channel].copy()
        hist_data = hist_data[['year_month_str', 'products__variants__sku', 'quantity']]

    # Only keep last 2 years of actuals (2024 + 2025) plus 2026-01
    hist_data = hist_data[
        (hist_data['year_month_str'] >= '2024-01') &
        (hist_data['year_month_str'] <= '2026-01')
    ].copy()

    hist_data = hist_data.merge(
        sku_details[['products__variants__sku', 'product_name']],
        on='products__variants__sku', how='left'
    )
    hist_data['type_flag'] = 'Actual'

    # --- Forecast (2026-02 to 2026-12) ---
    fcst_data = forecast_df[forecast_df['channel'] == channel][['month', 'sku', 'product_name', 'forecast_qty']].copy()
    fcst_data.columns = ['year_month_str', 'products__variants__sku', 'product_name', 'quantity']
    fcst_data['type_flag'] = 'Forecast'

    # Combine
    combined = pd.concat([hist_data, fcst_data], ignore_index=True)
    # Pivot with two index levels: product_name and sku
    combined['product_name'] = combined['product_name'].fillna('')
    combined['products__variants__sku'] = combined['products__variants__sku'].fillna('')

    pivot = combined.pivot_table(
        index=['product_name', 'products__variants__sku'],
        columns='year_month_str',
        values='quantity',
        fill_value=0,
        aggfunc='sum'
    )

    # Sort columns chronologically
    pivot = pivot.reindex(sorted(pivot.columns), axis=1)

    return pivot


comparison_pivots = {}
for channel in all_channels:
    comparison_pivots[channel] = build_comparison_pivot(channel, monthly_data, forecast_df, sku_details)

print("✅ Comparison pivots built (2024 actuals | 2025 actuals | 2026 forecast)")
# Show column range for verification
sample_cols = list(comparison_pivots['TOTAL'].columns)
print(f"Columns: {sample_cols[0]} → {sample_cols[-1]} ({len(sample_cols)} months)")

✅ Comparison pivots built (2024 actuals | 2025 actuals | 2026 forecast)
Columns: 2024-01 → 2026-12 (36 months)


In [13]:
# Create comparison DataFrames and helper structures
comparison_results = []
for channel in all_channels:
    channel_forecasts = forecast_df[forecast_df['channel'] == channel]
    total_channel_forecast = channel_forecasts['forecast_qty'].sum()
    product_type_forecasts = channel_forecasts.groupby('product_type')['forecast_qty'].sum().reset_index()
    total_by_product_type = product_type_forecasts['forecast_qty'].sum()
    total_by_sku = channel_forecasts.groupby('sku')['forecast_qty'].sum().sum()

    comparison_results.append({
        'Channel': channel,
        'A - Total Forecast': round(total_channel_forecast, 0),
        'B - Product Type': round(total_by_product_type, 0),
        'C - Item Level': round(total_by_sku, 0),
        'B vs A Diff': round(total_by_product_type - total_channel_forecast, 2),
        'C vs A Diff': round(total_by_sku - total_channel_forecast, 2)
    })

comparison_df = pd.DataFrame(comparison_results)

# Product type breakdown
product_type_details = []
for channel in all_channels:
    channel_forecasts = forecast_df[forecast_df['channel'] == channel]
    pt_summary = channel_forecasts.groupby('product_type').agg(
        Total_Forecast_Qty=('forecast_qty', 'sum'),
        Num_SKUs=('sku', 'nunique')
    ).reset_index()
    pt_summary.columns = ['Product_Type', 'Total_Forecast_Qty', 'Num_SKUs']
    pt_summary['Channel'] = channel
    pt_summary['Avg_Per_SKU'] = (pt_summary['Total_Forecast_Qty'] / pt_summary['Num_SKUs']).round(1)
    channel_total = pt_summary['Total_Forecast_Qty'].sum()
    pt_summary['Pct_of_Channel'] = ((pt_summary['Total_Forecast_Qty'] / channel_total) * 100).round(1)
    product_type_details.append(pt_summary)

product_type_df = pd.concat(product_type_details, ignore_index=True)
product_type_df = product_type_df[['Channel', 'Product_Type', 'Num_SKUs', 'Total_Forecast_Qty', 'Avg_Per_SKU', 'Pct_of_Channel']]

# Channel summary
summary_by_channel = []
for channel in all_channels:
    if channel == 'TOTAL':
        channel_monthly = monthly_data.groupby(['year_month_str'])['quantity'].sum().reset_index()
    else:
        channel_monthly = monthly_data[monthly_data['orders__source'] == channel].groupby(['year_month_str'])['quantity'].sum().reset_index()
    channel_monthly['year'] = channel_monthly['year_month_str'].str[:4]
    yearly = channel_monthly.groupby('year')['quantity'].sum()
    forecast_total = forecast_df[forecast_df['channel'] == channel]['forecast_qty'].sum()
    summary_by_channel.append({
        'Channel': channel,
        '2022_Total': int(yearly.get('2022', 0)),
        '2023_Total': int(yearly.get('2023', 0)),
        '2024_Total': int(yearly.get('2024', 0)),
        '2025_Total': int(yearly.get('2025', 0)),
        '2026_YTD': int(yearly.get('2026', 0)),
        '2026_Forecast': int(forecast_total)
    })

summary_df = pd.DataFrame(summary_by_channel)
summary_df['YoY_Growth'] = ((summary_df['2026_Forecast'] - summary_df['2025_Total']) / summary_df['2025_Total'] * 100).round(1)

print("✅ Summary dataframes ready")

✅ Summary dataframes ready


In [14]:
# ============================================================
# STEP 6: BUILD AGGREGATED VIEWS — per channel, monthly rows
# ============================================================
# Output structure per channel tab:
#   Product Name | SKU | Month | Year 2024 | Year 2025 | Forecast 2026
# Each SKU has 12 rows (one per calendar month Jan–Dec).
# Year 2024 / Year 2025 = sum of actuals for that month across both years.
# Forecast 2026 = Jan actual + Feb-Dec statistical forecast.
# A subtotal row is added per SKU, and a grand total at the bottom.
#
# Also builds the Building Blocks dataset:
#   Product Name | SKU | Month | Raw 2025 | Deseasonalized 2025 | Seasonal Index | Trend Slope | Forecast 2026
# ============================================================

MONTHS = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

def col_letter(n):
    """Convert 1-based column index to spreadsheet letter(s)."""
    result = ''
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        result = chr(65 + remainder) + result
    return result


def build_channel_monthly_view(channel, monthly_data, forecast_df, sku_details):
    """
    Returns a list of row dicts with columns:
      Product Name, SKU, Product_Type, Month (1-12), Month_Name,
      Year_2024, Year_2025, Forecast_2026
    One row per SKU per calendar month. Only SKUs with any data included.
    """
    # --- actuals ---
    if channel == 'TOTAL':
        act = monthly_data.groupby(['year_month_str','products__variants__sku'])['quantity'].sum().reset_index()
    else:
        act = monthly_data[monthly_data['orders__source'] == channel][
            ['year_month_str','products__variants__sku','quantity']].copy()

    act = act.merge(sku_details[['products__variants__sku','product_name','products__product_type']],
                    on='products__variants__sku', how='left')
    act['year']  = act['year_month_str'].str[:4].astype(int)
    act['month'] = act['year_month_str'].str[5:7].astype(int)

    # sum actuals by SKU + year + calendar month
    act_pivot = act.groupby(['products__variants__sku','product_name','products__product_type','year','month'])[
        'quantity'].sum().reset_index()

    # DEBUG: Check what years are in act_pivot
    if len(act_pivot) > 0:
        years_in_pivot = sorted(act_pivot['year'].unique())
        print(f"  DEBUG {channel}: act_pivot has {len(act_pivot)} rows, years: {years_in_pivot}")
        # Show sample for a SKU with 2025 data
        sample_2025 = act_pivot[act_pivot['year'] == 2025].head(3)
        if len(sample_2025) > 0:
            print(f"  DEBUG {channel}: Sample 2025 rows:")
            for _, row in sample_2025.iterrows():
                print(f"    {row['products__variants__sku']}: {row['year']}-{row['month']:02d} = {row['quantity']}")
    else:
        print(f"  DEBUG {channel}: WARNING - act_pivot is EMPTY!")

    # --- forecasts (Feb-Dec 2026) ---
    fcst = forecast_df[forecast_df['channel'] == channel][
        ['sku','product_name','product_type','month','forecast_qty']].copy()
    fcst['month_num'] = fcst['month'].str[5:7].astype(int)

    # --- build rows ---
    rows = []
    # Iterate over SKUs that have EITHER historical data OR forecast data
    skus_with_actuals = set(act_pivot['products__variants__sku'].unique())
    skus_with_forecast = set(fcst['sku'].unique())
    all_skus = list(skus_with_actuals | skus_with_forecast)

    # DEBUG: Check if FG-10103 is in the lists
    if "FG-10103" in skus_with_actuals:
        print(f"  DEBUG {channel}: FG-10103 is in skus_with_actuals")
    if "FG-10103" in skus_with_forecast:
        print(f"  DEBUG {channel}: FG-10103 is in skus_with_forecast")
    if "FG-10103" in all_skus:
        print(f"  DEBUG {channel}: FG-10103 is in all_skus (will be processed)")
    else:
        print(f"  DEBUG {channel}: FG-10103 is NOT in all_skus (will be skipped!)")

    for sku in all_skus:
        sku_info = sku_details[sku_details['products__variants__sku'] == sku]
        if len(sku_info) == 0:
            continue
        sku_info = sku_info.iloc[0]
        prod_name  = sku_info['product_name']
        prod_type  = sku_info['products__product_type']

        sku_act  = act_pivot[act_pivot['products__variants__sku'] == sku]
        sku_fcst = fcst[fcst['sku'] == sku]

        # Check if this SKU has any data for this channel at all
        # Use .isin() to handle both int and np.int64 types
        sku_2024 = sku_act[sku_act['year'].isin([2024])]
        sku_2025 = sku_act[sku_act['year'].isin([2025])]
        sku_2026_jan = sku_act[(sku_act['year'].isin([2026])) & (sku_act['month'].isin([1]))]
        has_data = (len(sku_2024) + len(sku_2025) + len(sku_2026_jan) + len(sku_fcst)) > 0
        if not has_data:
            continue

        # DEBUG: Show what we have for the first SKU only
        if sku == list(all_skus)[0]:
            print(f"  DEBUG: First SKU {sku}:")
            print(f"    sku_2024 rows: {len(sku_2024)}, sku_2025 rows: {len(sku_2025)}")
            if len(sku_2025) > 0:
                print(f"    sku_2025 months: {sorted(sku_2025["month"].unique())}")
                print(f"    sku_2025 total: {sku_2025["quantity"].sum()}")

        for m in range(1, 13):
            val_2024 = int(sku_2024[sku_2024['month'] == m]['quantity'].sum())
            val_2025 = int(sku_2025[sku_2025['month'] == m]['quantity'].sum())

            # DEBUG: Active debug for FG-10103
            if sku == "FG-10103":
                if m == 1:  # Only print once per SKU
                    print(f"\n  DEBUG SKU {sku} in {channel}:")
                    print(f"    sku_2024 rows: {len(sku_2024)}, sku_2025 rows: {len(sku_2025)}")
                    if len(sku_2025) > 0:
                        print(f"    sku_2025 months available: {sorted(sku_2025["month"].unique())}")
                        print(f"    sku_2025 total quantity: {sku_2025["quantity"].sum()}")
                    else:
                        print(f"    sku_2025 is EMPTY for this channel")
                if val_2025 == 0 and len(sku_2025) > 0:
                    print(f"    Month {m}: val_2025=0 BUT sku_2025 has {len(sku_2025)} rows!")

            # 2026: Jan = actual, Feb-Dec = forecast
            if m == 1:
                val_2026 = int(sku_2026_jan['quantity'].sum()) if len(sku_2026_jan) > 0 else 0
            else:
                match = sku_fcst[sku_fcst['month_num'] == m]
                val_2026 = int(match['forecast_qty'].sum()) if len(match) > 0 else 0

            is_active = sku in active_skus_from_file

            rows.append({
                'Product Name': prod_name,
                'SKU': sku,
                'Product_Type': prod_type,
                'Is_Active': 'Yes' if is_active else 'No',
                'Month': m,
                'Month_Name': MONTHS[m-1],
                'Year_2024': val_2024,
                'Year_2025': val_2025,
                'Forecast_2026': val_2026,
            })

    return pd.DataFrame(rows)


def build_building_blocks(monthly_data, forecast_df, sku_details, pt_seasonal_indices, channel_growth_rates=None):
    """
    For each active SKU, for each calendar month, shows:
      Product Name | SKU | Product_Type | Month | Month_Name |
      Raw_2025 | Seasonal_Index | Deseasonalized_2025 |
      Trend_Slope_Per_Month | Base_Level | Forecast_2026
    """
    # TOTAL channel actuals
    act = monthly_data.groupby(['year_month_str','products__variants__sku'])['quantity'].sum().reset_index()
    act['year']  = act['year_month_str'].str[:4].astype(int)
    act['month'] = act['year_month_str'].str[5:7].astype(int)
    act_2025 = act[act['year'] == 2025].copy()

    # forecasts (TOTAL channel)
    fcst = forecast_df[forecast_df['channel'] == 'TOTAL'][
        ['sku','month','forecast_qty']].copy()
    fcst['month_num'] = fcst['month'].str[5:7].astype(int)

    if channel_growth_rates is None:
        channel_growth_rates = {}

    rows = []
    for sku in sku_details['products__variants__sku'].unique():

        sku_info = sku_details[sku_details['products__variants__sku'] == sku]
        if len(sku_info) == 0:
            continue
        sku_info  = sku_info.iloc[0]
        prod_name = sku_info['product_name']
        prod_type = sku_info['products__product_type']
        seas_idx  = pt_seasonal_indices.get(prod_type, {m: 1.0 for m in range(1,13)})

        # Get historical values for trend/base calculation (same as forecast engine)
        sku_hist = act[(act['products__variants__sku'] == sku) &
                       (act['year_month_str'] <= '2026-01')].sort_values('year_month_str')
        hist_vals  = sku_hist['quantity'].values.astype(float)
        hist_months = np.array([int(ym[5:7]) for ym in sku_hist['year_month_str'].values])

        n = len(hist_vals)
        window = min(n, 12)
        if window >= 4:
            recent_vals   = hist_vals[-window:]
            recent_months = hist_months[-window:]
            deseason_recent = np.array([
                v / seas_idx.get(int(m), 1.0) if seas_idx.get(int(m), 1.0) > 0 else v
                for v, m in zip(recent_vals, recent_months)
            ])
            # exponential weighted base
            alpha = 0.15
            wts = np.array([(1 - alpha)**(window - 1 - i) for i in range(window)])
            base_level  = float(np.dot(wts, deseason_recent) / wts.sum())
            # weighted trend slope
            weights = np.arange(1, window + 1, dtype=float)
            x = np.arange(window, dtype=float)
            wsum = weights.sum(); wx = (weights*x).sum(); wy = (weights*deseason_recent).sum()
            wxx = (weights*x*x).sum(); wxy = (weights*x*deseason_recent).sum()
            denom = wsum*wxx - wx*wx
            trend_slope = float((wsum*wxy - wx*wy) / denom) if abs(denom) > 1e-10 else 0.0
            max_slope = base_level * 0.025 if base_level > 0 else 1.0
            trend_slope = float(np.clip(trend_slope, -max_slope, max_slope))
        else:
            base_level  = float(np.mean(hist_vals)) if n > 0 else 0.0
            trend_slope = 0.0

        sku_2025_act = act_2025[act_2025['products__variants__sku'] == sku]
        sku_fcst     = fcst[fcst['sku'] == sku]

        for m in range(1, 13):
            raw_2025 = int(sku_2025_act[sku_2025_act['month'] == m]['quantity'].sum())
            si = round(seas_idx.get(m, 1.0), 4)
            deseas_2025 = round(raw_2025 / si, 1) if si > 0 else raw_2025

            match = sku_fcst[sku_fcst['month_num'] == m]
            fcst_2026 = int(match['forecast_qty'].sum()) if len(match) > 0 else 0
            if m == 1:
                jan_act = act[(act['products__variants__sku'] == sku) &
                              (act['year'] == 2026) & (act['month'] == 1)]['quantity'].sum()
                fcst_2026 = int(jan_act)

            rows.append({
                'Product Name': prod_name,
                'SKU': sku,
                'Product_Type': prod_type,
                'Month': m,
                'Month_Name': MONTHS[m-1],
                'Raw_2025_Actual': raw_2025,
                'Seasonal_Index': si,
                'Deseasonalized_2025': deseas_2025,
                'Base_Level_(deseason)': round(base_level, 1),
                'Trend_Slope_(units/mo)': round(trend_slope, 4),
                'Growth_Rate_Override': {ch: f'{r*100:+.1f}%' for ch, r in channel_growth_rates.items()}.get('TOTAL', '+0.0%'),
                'DTC_Growth_Override':  f"{channel_growth_rates.get('Direct-to-Consumer', 0.0)*100:+.1f}%",
                'Wholesale_Growth_Override': f"{channel_growth_rates.get('Wholesale', 0.0)*100:+.1f}%",
                'Total_Growth_Override': f"{channel_growth_rates.get('TOTAL', 0.0)*100:+.1f}%",
                'Forecast_2026': fcst_2026,
            })

    return pd.DataFrame(rows)


# Build all views
print("Building per-channel monthly views...")
channel_monthly_views = {}
for ch in all_channels:
    channel_monthly_views[ch] = build_channel_monthly_view(
        ch, monthly_data, forecast_df, sku_details)
    n_skus = channel_monthly_views[ch]['SKU'].nunique()
    print(f"  {ch}: {n_skus} SKUs × 12 months = {len(channel_monthly_views[ch])} rows")

print("\nBuilding building blocks view...")
building_blocks_df = build_building_blocks(
    monthly_data, forecast_df, sku_details, pt_seasonal_indices,
    channel_growth_rates=CHANNEL_GROWTH_RATES)
print(f"  Building Blocks: {len(building_blocks_df)} rows ({building_blocks_df['SKU'].nunique()} active SKUs)")
print("\n✅ All views ready")


Building per-channel monthly views...
  DEBUG Direct-to-Consumer: act_pivot has 3404 rows, years: [np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025), np.int64(2026)]
  DEBUG Direct-to-Consumer: Sample 2025 rows:
    BLANKET-W-1INSERT: 2025-11 = 1
    BLANKET-W-3INSERT: 2025-02 = 1
    FG-10002: 2025-01 = 11
  DEBUG Direct-to-Consumer: FG-10103 is in skus_with_forecast
  DEBUG Direct-to-Consumer: FG-10103 is in all_skus (will be processed)
  DEBUG: First SKU FG-110001:
    sku_2024 rows: 8, sku_2025 rows: 11
    sku_2025 months: [np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12)]
    sku_2025 total: 25

  DEBUG SKU FG-10103 in Direct-to-Consumer:
    sku_2024 rows: 0, sku_2025 rows: 0
    sku_2025 is EMPTY for this channel
  Direct-to-Consumer: 156 SKUs × 12 months = 1872 rows
  DEBUG Wholesale: act_pivot has 3740 rows, years: [np.int64(2022), np.int64(2023), np.int64(2024), np.in

In [17]:
# Authenticate with Google and create new Google Sheet
import gspread
from google.colab import auth
from google.auth import default

print("Authenticating with Google...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ Authenticated")

Authenticating with Google...
✅ Authenticated


In [18]:
# Create new Google Sheet
sheet_name = f"Demand_Planning_{datetime.now().strftime('%Y%m%d_%H%M')}"
sh = gc.create(sheet_name)
print(f"Created Google Sheet: {sheet_name}")
print(f"URL: https://docs.google.com/spreadsheets/d/{sh.id}")

Created Google Sheet: Demand_Planning_20260218_1622
URL: https://docs.google.com/spreadsheets/d/18330-VAQg6Cd-kWFujImQ8agIyUwqH_qWRzb6_obE6k


In [19]:
# ============================================================
# WRITE: Summary Dashboard
# ============================================================
summary_sheet = sh.sheet1
summary_sheet.update_title('Summary Dashboard')

summary_sheet.update('A1', [['DEMAND PLANNING SUMMARY']])
summary_sheet.update('A3', [['Historical Period: 2022-01 through 2026-01']])
summary_sheet.update('A4', [['Forecast Period: 2026-02 through 2026-12']])

# --- Methodology explanation ---
method_rows = [
    ['FORECASTING METHODOLOGY'],
    [''],
    ['SEASONALITY (Product-Type Level)'],
    ['  Seasonal indices are calculated by pooling all SKUs within each product type together, using the last 24 months'],
    ['  of TOTAL channel sales data. For each calendar month (Jan–Dec), we compute the average volume relative to the'],
    ['  overall monthly average for that product type. This produces a stable index (e.g. 1.42 = 42% above average,'],
    ['  0.71 = 29% below average) that is shared by all SKUs in the same product type. Using product-type pooling'],
    ['  rather than per-item indices prevents noisy, low-volume SKUs from generating unreliable seasonal patterns.'],
    [''],
    ['TREND (Per-SKU, Recency-Weighted)'],
    ['  A linear trend is estimated for each SKU individually using the last 12 months of deseasonalized sales.'],
    ['  Weighted least-squares regression is used, where the most recent month carries 12x the weight of the'],
    ['  oldest month in the window. This means recent acceleration or deceleration in demand has a much stronger'],
    ['  influence on the slope than older data. The trend slope is capped at ±2.5% of the base level per month'],
    ['  (~30% annually) to prevent runaway projections on sparse or erratic SKUs.'],
    [''],
    ['BASE LEVEL (Per-SKU)'],
    ['  The deseasonalized base level is calculated as an exponentially weighted moving average of the last 12'],
    ['  months (decay factor alpha=0.15). Recent months carry significantly more weight than older months,'],
    ['  so the base level responds to recent demand shifts while remaining stable against one-off spikes.'],
    [''],
    ['FORECAST CALCULATION'],
    ['  For each future month: Forecast = (Base Level + Trend × Steps Ahead) × Seasonal Index × Growth Factor'],
    ['  Seasonal Index: product-type index for that calendar month'],
    ['  Growth Factor: optional manual override (default 0% = pure statistical forecast)'],
    ['  All forecasts are rounded to whole units — no fractional quantities.'],
    [''],
    ['INACTIVITY RULE'],
    ['  Any SKU with zero total sales across all channels in the 6-month window prior to the forecast start'],
    ['  is classified as inactive and receives no forecast. These SKUs still appear in historical views.'],
    [''],
    ['WHAT THIS FORECAST DOES NOT CAPTURE'],
    ['  The statistical model extrapolates demand patterns from historical sell-through data. It does not'],
    ['  account for the following factors, which should be applied as manual judgment on top of the forecast:'],
    ['  • Promotions & Discounts: Heavy discounting (e.g. sitewide sales) inflates historical volume in'],
    ['    those months. The model will partially absorb this into the base level and seasonal index,'],
    ['    which can cause future months to be over- or under-forecast relative to promo intent.'],
    ['  • New Product Launches: SKUs with < 6 months of history have limited trend signal. Review'],
    ['    their forecasts manually and consider applying a growth override.'],
    ['  • Planned Price Changes: A price increase typically suppresses volume; a reduction lifts it.'],
    ['    Neither is visible to the model.'],
    ['  • Inventory / Supply Constraints: Stockouts in the historical window appear as zero demand,'],
    ['    causing the model to underestimate true underlying demand for those periods.'],
    ['  • Channel Mix Shifts: If volume is intentionally being moved between DTC and Wholesale,'],
    ['    the channel-level forecasts will not reflect that intent.'],
    ['  • Discontinued SKUs: Captured by the inactivity rule but only if sales went to zero in the'],
    ['    last 6 months. SKUs being wound down gradually will still receive a (likely too-high) forecast.'],
    [''],
]
summary_sheet.update('A6', method_rows)

method_end_row = 6 + len(method_rows) + 1
summary_sheet.update(f'A{method_end_row}', [['CHANNEL SUMMARY']])
summary_data = [summary_df.columns.tolist()] + summary_df.values.tolist()
summary_sheet.update(f'A{method_end_row + 2}', summary_data)

comparison_start = method_end_row + 12
summary_sheet.update(f'A{comparison_start}', [['FORECAST AGGREGATION COMPARISON']])
comparison_data = [comparison_df.columns.tolist()] + comparison_df.values.tolist()
summary_sheet.update(f'A{comparison_start + 2}', comparison_data)

summary_sheet.format('A1', {'textFormat': {'bold': True, 'fontSize': 14}})
summary_sheet.format('A6', {'textFormat': {'bold': True, 'fontSize': 12}})
summary_sheet.format('A8', {'textFormat': {'bold': True, 'underline': True}})
summary_sheet.format('A14', {'textFormat': {'bold': True, 'underline': True}})
summary_sheet.format('A21', {'textFormat': {'bold': True, 'underline': True}})
summary_sheet.format('A27', {'textFormat': {'bold': True, 'underline': True}})
summary_sheet.format('A33', {'textFormat': {'bold': True, 'underline': True}})
summary_sheet.format(f'A{method_end_row}', {'textFormat': {'bold': True, 'fontSize': 12}})
summary_sheet.format(f'A{method_end_row + 2}:H{method_end_row + 2}', {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
})
summary_sheet.format(f'A{comparison_start}', {'textFormat': {'bold': True, 'fontSize': 12}})
summary_sheet.format(f'A{comparison_start + 2}:F{comparison_start + 2}', {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
})

print("Summary Dashboard created with full methodology explanation")

Summary Dashboard created with full methodology explanation


In [20]:
# ============================================================
# WRITE: Product Type Breakdown (with Seasonality Table)
# ============================================================
pt_sheet = sh.add_worksheet(title='Product Type Breakdown', rows=1000, cols=30)

# Section 1: Seasonality indices
pt_sheet.update('A1', [['SEASONAL INDICES BY PRODUCT TYPE AND MONTH']])
pt_sheet.update('A2', [['Based on last 24 months of TOTAL channel data. Index > 1.0 = stronger than average. Normalized so each row averages to 1.0.']])

seas_data = [seasonality_display_df.columns.tolist()] + seasonality_display_df.values.tolist()
pt_sheet.update('A4', seas_data)

# Header formatting for seasonality table
pt_sheet.format('A1', {'textFormat': {'bold': True, 'fontSize': 13}})
num_pt_rows = len(seasonality_display_df)
header_range = f'A4:M4'
pt_sheet.format(header_range, {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.18, 'green': 0.53, 'blue': 0.33}  # green
})

# Section 2: Forecast summary by product type
sep_row = num_pt_rows + 7  # leave a gap
pt_sheet.update(f'A{sep_row}', [['FORECAST SUMMARY BY PRODUCT TYPE AND CHANNEL']])
pt_sheet.format(f'A{sep_row}', {'textFormat': {'bold': True, 'fontSize': 13}})

pt_data = [product_type_df.columns.tolist()] + product_type_df.values.tolist()
pt_sheet.update(f'A{sep_row + 2}', pt_data)
pt_sheet.format(f'A{sep_row + 2}:F{sep_row + 2}', {
    'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
    'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
})

print("Product Type Breakdown sheet created (with seasonality indices)")

Product Type Breakdown sheet created (with seasonality indices)


In [21]:
# ============================================================
# WRITE: Channel Forecast + Comparison Pivot sheets
# Each channel gets ONE sheet: 2024 actuals | 2025 actuals | 2026 (Jan actual + Feb-Dec forecast)
# ============================================================

FORECAST_START = '2026-02'  # first forecast month

for channel in all_channels:
    print(f"Creating forecast sheet for {channel}...")

    ws = sh.add_worksheet(title=f"{channel} - Forecast", rows=2000, cols=150)

    pivot = comparison_pivots[channel]
    all_cols = list(pivot.columns)  # chronologically sorted month strings

    # Identify which columns are forecast vs actual
    # 2026-01 = actual, 2026-02 onward = forecast
    actual_cols = [c for c in all_cols if c < FORECAST_START]
    forecast_cols = [c for c in all_cols if c >= FORECAST_START]

    # Build header rows
    # Row 1: title
    ws.update('A1', [[f'{channel} — Historical vs Forecast']])

    # Row 2: year group labels  (2024 actuals / 2025 actuals / 2026 actual+forecast)
    year_label_row = ['Product Name', 'SKU']
    prev_year = None
    col_labels = []
    for c in all_cols:
        yr = c[:4]
        is_fcst = (c >= FORECAST_START)
        label = f"{yr} {'[FORECAST]' if is_fcst else '[ACTUAL]'}"
        col_labels.append(label)
    year_label_row = ['Product Name', 'SKU'] + col_labels

    # Row 3: month column headers
    month_header_row = ['Product Name', 'SKU'] + all_cols

    # Data rows — MultiIndex: (product_name, sku)
    data_rows = []
    for idx in pivot.index:
        prod_name, sku_code = idx
        row = [prod_name, sku_code] + [int(round(pivot.loc[idx, c])) if c in pivot.columns else 0 for c in all_cols]
        data_rows.append(row)

    # Add a TOTALS row
    totals_row_data = ['** CHANNEL TOTAL **', '']
    for c in all_cols:
        if c in pivot.columns:
            totals_row_data.append(int(round(pivot[c].sum())))
        else:
            totals_row_data.append(0)
    data_rows.append(totals_row_data)

    ws.update('A2', [year_label_row])
    ws.update('A3', [month_header_row])
    ws.update('A4', data_rows)

    # Format headers
    ws.format('A1', {'textFormat': {'bold': True, 'fontSize': 13}})

    # Row 2: alternating year/type shading
    # Row 3: column month headers — bold
    ws.format('A3', {'textFormat': {'bold': True}})

    # Color the actual columns header (row 2) in grey-blue
    num_actual = len(actual_cols)
    num_forecast = len(forecast_cols)

    if num_actual > 0:
        # Columns B onward for actual (1-indexed → col B = 2)
        import string
        def col_letter(n):
            """Convert 1-based column index to letter(s)."""
            result = ''
            while n > 0:
                n, remainder = divmod(n - 1, 26)
                result = chr(65 + remainder) + result
            return result

        actual_start_col = col_letter(2)  # B
        actual_end_col = col_letter(1 + num_actual)
        ws.format(f'{actual_start_col}2:{actual_end_col}2', {
            'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
            'backgroundColor': {'red': 0.36, 'green': 0.44, 'blue': 0.56}  # slate
        })
        ws.format(f'{actual_start_col}3:{actual_end_col}3', {
            'textFormat': {'bold': True}
        })

    if num_forecast > 0:
        fcst_start_col = col_letter(2 + num_actual)
        fcst_end_col = col_letter(1 + num_actual + num_forecast)
        ws.format(f'{fcst_start_col}2:{fcst_end_col}2', {
            'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
            'backgroundColor': {'red': 0.18, 'green': 0.53, 'blue': 0.33}  # green for forecast
        })
        ws.format(f'{fcst_start_col}3:{fcst_end_col}3', {
            'textFormat': {'bold': True, 'foregroundColor': {'red': 0.1, 'green': 0.5, 'blue': 0.2}}
        })

    print(f"  → {len(all_cols)} months ({num_actual} actuals + {num_forecast} forecast)")

print("\nAll channel forecast sheets created")

Creating forecast sheet for Direct-to-Consumer...
  → 36 months (25 actuals + 11 forecast)
Creating forecast sheet for Wholesale...
  → 36 months (25 actuals + 11 forecast)
Creating forecast sheet for TOTAL...
  → 36 months (25 actuals + 11 forecast)

All channel forecast sheets created


In [22]:
# ============================================================
# WRITE: Aggregated By Channel (3 tabs) + Building Blocks
# ============================================================

def write_channel_agg_sheet(sh, title, df):
    """
    Write a channel aggregated sheet.
    Layout: Product Name | SKU | Month | Year 2024 | Year 2025 | Forecast 2026
    Each SKU spans 12 rows (one per month) followed by a subtotal row.
    Grand total at the bottom.
    """
    ws = sh.add_worksheet(title=title, rows=5000, cols=10)

    header = ['Product Name', 'SKU', 'Product Type', 'Is Active', 'Month', 'Year 2024', 'Year 2025', 'Forecast 2026']
    data_rows = [header]

    skus_in_order = df['SKU'].unique()
    grand = {'Year_2024': 0, 'Year_2025': 0, 'Forecast_2026': 0}

    for sku in skus_in_order:
        sku_df = df[df['SKU'] == sku].sort_values('Month')
        prod_name = sku_df['Product Name'].iloc[0]
        prod_type = str(sku_df['Product_Type'].iloc[0]) if pd.notna(sku_df['Product_Type'].iloc[0]) else ''
        is_active = sku_df['Is_Active'].iloc[0]
        tot_2024 = sku_df['Year_2024'].sum()
        tot_2025 = sku_df['Year_2025'].sum()
        tot_2026 = sku_df['Forecast_2026'].sum()

        for _, row in sku_df.iterrows():
            data_rows.append([
                prod_name, sku, prod_type, is_active,
                row['Month_Name'],
                row['Year_2024'], row['Year_2025'], row['Forecast_2026']
            ])

        # Subtotal row for this SKU
        data_rows.append([prod_name + ' — TOTAL', sku, prod_type, is_active, 'ANNUAL',
                          int(tot_2024), int(tot_2025), int(tot_2026)])
        data_rows.append(['', '', '', '', '', '', '', ''])  # spacer

        grand['Year_2024']     += tot_2024
        grand['Year_2025']     += tot_2025
        grand['Forecast_2026'] += tot_2026

    # Grand total
    data_rows.append(['** GRAND TOTAL **', '', '', '', '',
                      int(grand["Year_2024"]), int(grand["Year_2025"]), int(grand["Forecast_2026"])])

    ws.update('A1', data_rows)

    # Formatting
    num_rows = len(data_rows)
    ws.format('A1:H1', {
        'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
        'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
    })
    ws.format(f'A{num_rows}:H{num_rows}', {
        'textFormat': {'bold': True},
        'backgroundColor': {'red': 0.95, 'green': 0.95, 'blue': 0.75}
    })

    # Color Year 2024 header grey, Year 2025 slate, Forecast 2026 green
    ws.format('F1', {'backgroundColor': {'red': 0.75, 'green': 0.75, 'blue': 0.75}})
    ws.format('G1', {'backgroundColor': {'red': 0.36, 'green': 0.44, 'blue': 0.56},
                     'textFormat': {'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}, 'bold': True}})
    ws.format('H1', {'backgroundColor': {'red': 0.18, 'green': 0.53, 'blue': 0.33},
                     'textFormat': {'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}, 'bold': True}})

    print(f"  ✅ {title}: {len(data_rows)} rows written")
    return ws


def write_building_blocks_sheet(sh, df):
    """
    Write the Building Blocks sheet showing deseasonalization components.
    Columns: Product Name | SKU | Product Type | Month | Month_Name |
             Raw 2025 Actual | Seasonal Index | Deseasonalized 2025 |
             Base Level | Trend Slope/Month | Forecast 2026
    """
    ws = sh.add_worksheet(title='Building Blocks', rows=5000, cols=15)

    header = [
        'Product Name', 'SKU', 'Product Type', 'Month #', 'Month',
        'Raw 2025 Actual', 'Seasonal Index', 'Deseasonalized 2025',
        'Base Level (deseason)', 'Trend Slope (units/mo)',
        'DTC Growth Override', 'Wholesale Growth Override', 'Total Growth Override',
        'Forecast 2026'
    ]

    data_rows = [header]
    skus = df['SKU'].unique()

    for sku in skus:
        sku_df = df[df['SKU'] == sku].sort_values('Month')
        prod_name = sku_df['Product Name'].iloc[0]
        prod_type = str(sku_df['Product_Type'].iloc[0]) if pd.notna(sku_df['Product_Type'].iloc[0]) else ''

        for _, row in sku_df.iterrows():
            data_rows.append([
                prod_name, sku, prod_type,
                int(row['Month']), row['Month_Name'],
                row['Raw_2025_Actual'],
                row['Seasonal_Index'],
                row['Deseasonalized_2025'],
                row['Base_Level_(deseason)'],
                row['Trend_Slope_(units/mo)'],
                row['DTC_Growth_Override'],
                row['Wholesale_Growth_Override'],
                row['Total_Growth_Override'],
                row['Forecast_2026'],
            ])
        data_rows.append(['', '', '', '', '', '', '', '', '', '', ''])  # spacer

    ws.update('A1', data_rows)

    ws.format('A1:N1', {
        'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
        'backgroundColor': {'red': 0.18, 'green': 0.33, 'blue': 0.53}  # deep blue
    })
    # Colour the three building-block columns differently
    ws.format('F1', {'backgroundColor': {'red': 0.36, 'green': 0.44, 'blue': 0.56}})  # raw = slate
    ws.format('G1', {'backgroundColor': {'red': 0.60, 'green': 0.40, 'blue': 0.70}})  # index = purple
    ws.format('H1', {'backgroundColor': {'red': 0.60, 'green': 0.40, 'blue': 0.70}})  # deseas = purple
    ws.format('I1', {'backgroundColor': {'red': 0.85, 'green': 0.65, 'blue': 0.13}})  # base = amber
    ws.format('J1', {'backgroundColor': {'red': 0.85, 'green': 0.65, 'blue': 0.13}})  # trend = amber
    ws.format('K1:M1', {
        'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
        'backgroundColor': {'red': 0.82, 'green': 0.35, 'blue': 0.20}  # burnt orange = override
    })
    ws.format('N1', {'backgroundColor': {'red': 0.18, 'green': 0.53, 'blue': 0.33}})  # forecast = green

    print(f"  ✅ Building Blocks: {len(data_rows)} rows written")
    return ws


# Write the three channel tabs
print("Writing Aggregated By Channel sheets...")

# DEBUG: Check what's in the dataframes before writing
for ch in all_channels:
    df_check = channel_monthly_views[ch]
    if len(df_check) > 0:
        # Check a specific SKU that should have 2025 data
        sample_sku = df_check["SKU"].iloc[0]
        sample_data = df_check[df_check["SKU"] == sample_sku]
        year_2025_sum = sample_data["Year_2025"].sum()
        year_2024_sum = sample_data["Year_2024"].sum()
        print(f"\n  DEBUG {ch} dataframe check:")
        print(f"    Sample SKU: {sample_sku}")
        print(f"    Year_2024 total: {year_2024_sum}")
        print(f"    Year_2025 total: {year_2025_sum}")
        print(f"    Year_2025 sample values: {sample_data["Year_2025"].head(3).tolist()}")

for ch in all_channels:
    write_channel_agg_sheet(sh, f"Agg by Month — {ch}", channel_monthly_views[ch])

# Write Building Blocks tab
print("\nWriting Building Blocks sheet...")
write_building_blocks_sheet(sh, building_blocks_df)

print("\n✅ All aggregated sheets created")


Writing Aggregated By Channel sheets...

  DEBUG Direct-to-Consumer dataframe check:
    Sample SKU: FG-110001
    Year_2024 total: 22
    Year_2025 total: 25
    Year_2025 sample values: [0, 4, 1]

  DEBUG Wholesale dataframe check:
    Sample SKU: FG-110001
    Year_2024 total: 5
    Year_2025 total: 18
    Year_2025 sample values: [0, 1, 1]

  DEBUG TOTAL dataframe check:
    Sample SKU: FG-110001
    Year_2024 total: 27
    Year_2025 total: 43
    Year_2025 sample values: [0, 5, 2]
  ✅ Agg by Month — Direct-to-Consumer: 2186 rows written
  ✅ Agg by Month — Wholesale: 1780 rows written
  ✅ Agg by Month — TOTAL: 2886 rows written

Writing Building Blocks sheet...
  ✅ Building Blocks: 3576 rows written

✅ All aggregated sheets created


In [23]:
# ============================================================
# BUILD: Aggregated By Year (for QC comparison)
# ============================================================

agg_by_year = []

# Aggregate monthly_data to yearly totals per SKU
total_monthly = monthly_data.groupby(['year_month_str', 'products__variants__sku'])['quantity'].sum().reset_index()
total_monthly['year'] = total_monthly['year_month_str'].str[:4]

for sku in sku_details['products__variants__sku'].unique():
    sku_info = sku_details[sku_details['products__variants__sku'] == sku].iloc[0]
    sku_data = total_monthly[total_monthly['products__variants__sku'] == sku]

    yearly = sku_data.groupby('year')['quantity'].sum()

    # 2026: Jan YTD actual
    ytd_2026 = int(sku_data[sku_data['year_month_str'] == '2026-01']['quantity'].sum())

    # 2026 Forecast: sum from forecast_df
    sku_fcst_2026 = forecast_df[
        (forecast_df['sku'] == sku) & (forecast_df['channel'] == 'TOTAL')
    ]['forecast_qty'].sum()

    total_2026 = ytd_2026 + int(sku_fcst_2026)

    is_active = sku in active_skus_from_file

    agg_by_year.append({
        'SKU': sku,
        'Product_Name': sku_info['product_name'],
        'Is_Active': 'Yes' if is_active else 'No',
        '2024': int(yearly.get('2024', 0)),
        '2025': int(yearly.get('2025', 0)),
        '2026_YTD': ytd_2026,
        '2026_Forecast': total_2026
    })

agg_by_year_df = pd.DataFrame(agg_by_year)
print(f"\n✅ Aggregated By Year built: {len(agg_by_year_df)} SKUs")
print(f"   Columns: {list(agg_by_year_df.columns)}")



✅ Aggregated By Year built: 275 SKUs
   Columns: ['SKU', 'Product_Name', 'Is_Active', '2024', '2025', '2026_YTD', '2026_Forecast']


In [24]:
# ============================================================
# UPLOAD: QC Check CSV
# ============================================================
# Upload a CSV with columns: finished_good, sales_2024, sales_2025, sales_2026
# This will be compared against the forecast data for quality checking

print("📂 Upload your QC check CSV file")
print("   Expected columns: finished_good, sales_2024, sales_2025, sales_2026")
print()

try:
    qc_files = files.upload()
    if qc_files:
        qc_filename = list(qc_files.keys())[0]
        print(f"\nLoading QC data from: {qc_filename}")

        qc_data = pd.read_csv(qc_filename)

        # Rename columns to match
        qc_data = qc_data.rename(columns={'finished_good': 'SKU'})

        print(f"✅ QC data loaded: {len(qc_data)} SKUs")
        print(f"   Columns: {list(qc_data.columns)}")
        print(f"   Sample SKUs: {qc_data["SKU"].head(3).tolist()}")
    else:
        qc_data = None
        print("⚠️  No QC file uploaded - skipping QC check")
except Exception as e:
    print(f"⚠️  Error loading QC file: {e}")
    qc_data = None


📂 Upload your QC check CSV file
   Expected columns: finished_good, sales_2024, sales_2025, sales_2026



Saving active items - qc check.csv to active items - qc check.csv

Loading QC data from: active items - qc check.csv
✅ QC data loaded: 57 SKUs
   Columns: ['SKU', 'sales_2024', 'sales_2025', 'sales_2026']
   Sample SKUs: ['FG-10004', 'FG-10005', 'FG-10006']


In [25]:
# ============================================================
# QC COMPARISON & SHEET CREATION
# ============================================================

if qc_data is not None:
    print("\nBuilding QC comparison...")

    # Merge forecast data with QC data
    qc_comparison = agg_by_year_df.merge(
        qc_data,
        on='SKU',
        how='outer',
        suffixes=('', '_QC')
    )

    # Calculate deltas
    qc_comparison['Delta_2024'] = qc_comparison['2024'] - qc_comparison['sales_2024'].fillna(0)
    qc_comparison['Delta_2025'] = qc_comparison['2025'] - qc_comparison['sales_2025'].fillna(0)
    qc_comparison['Delta_2026_YTD'] = qc_comparison['2026_YTD'] - qc_comparison['sales_2026'].fillna(0)

    # Reorder columns for clarity
    qc_comparison = qc_comparison[[
        'SKU', 'Product_Name', 'Is_Active',
        '2024', 'sales_2024', 'Delta_2024',
        '2025', 'sales_2025', 'Delta_2025',
        '2026_YTD', 'sales_2026', 'Delta_2026_YTD',
        '2026_Forecast'
    ]]

    # Fill NaN with 0 for cleaner display
    qc_comparison = qc_comparison.fillna(0).astype({
        '2024': int, 'sales_2024': int, 'Delta_2024': int,
        '2025': int, 'sales_2025': int, 'Delta_2025': int,
        '2026_YTD': int, 'sales_2026': int, 'Delta_2026_YTD': int,
        '2026_Forecast': int
    })

    print(f"✅ QC comparison built: {len(qc_comparison)} SKUs")
    print(f"\nSummary:")
    print(f"  Total Delta 2024: {qc_comparison['Delta_2024'].sum():,}")
    print(f"  Total Delta 2025: {qc_comparison['Delta_2025'].sum():,}")
    print(f"  Total Delta 2026 YTD: {qc_comparison['Delta_2026_YTD'].sum():,}")

    # Write QC sheet
    print("\nWriting QC Check sheet...")
    ws_qc = sh.add_worksheet(title='QC Check', rows=1000, cols=15)

    header = [
        'SKU', 'Product Name', 'Is Active',
        'Forecast 2024', 'QC 2024', 'Delta 2024',
        'Forecast 2025', 'QC 2025', 'Delta 2025',
        'Forecast 2026 YTD', 'QC 2026', 'Delta 2026 YTD',
        'Forecast 2026 Total'
    ]

    data_rows = [header] + qc_comparison.values.tolist()
    ws_qc.update('A1', data_rows)

    # Format header
    ws_qc.format('A1:M1', {
        'textFormat': {'bold': True, 'foregroundColor': {'red': 1, 'green': 1, 'blue': 1}},
        'backgroundColor': {'red': 0.259, 'green': 0.522, 'blue': 0.957}
    })

    # Highlight delta columns in yellow
    ws_qc.format('F:F', {'backgroundColor': {'red': 1, 'green': 1, 'blue': 0.8}})  # Delta 2024
    ws_qc.format('I:I', {'backgroundColor': {'red': 1, 'green': 1, 'blue': 0.8}})  # Delta 2025
    ws_qc.format('L:L', {'backgroundColor': {'red': 1, 'green': 1, 'blue': 0.8}})  # Delta 2026 YTD

    print(f"  ✅ QC Check sheet created with {len(data_rows)} rows")
else:
    print("\n⚠️  Skipping QC comparison (no QC data uploaded)")



Building QC comparison...
✅ QC comparison built: 275 SKUs

Summary:
  Total Delta 2024: 22,286
  Total Delta 2025: 15,879
  Total Delta 2026 YTD: -4,380

Writing QC Check sheet...
  ✅ QC Check sheet created with 276 rows


In [26]:
# Final output
print("\n" + "="*80)
print("DEMAND PLANNING TOOL v3 CREATED SUCCESSFULLY!")
print("="*80)
print(f"\nGoogle Sheet Name: {sheet_name}")
print(f"URL: https://docs.google.com/spreadsheets/d/{sh.id}")
print(f"\nSheets created:")
for worksheet in sh.worksheets():
    print(f"  - {worksheet.title}")
print(f"\nTotal SKUs analyzed: {len(sku_details)}")
print(f"Channels: {', '.join(all_channels)}")
print(f"Forecast period: Feb 2026 - Dec 2026")
print(f"\nWhat's new in v3:")
print(f"  ✅ Seasonality by PRODUCT TYPE (24-month pooled indices, not per-item)")
print(f"  ✅ Seasonality table in Product Type Breakdown tab (month × product type)")
print(f"  ✅ Forecast sheets show 2024 + 2025 actuals alongside 2026 forecast")
print(f"  ✅ Actual vs Forecast columns color-coded (slate = actual, green = forecast)")
print(f"  ✅ Channel total row added to each forecast sheet")
print(f"  ✅ Aggregated By Month — 3 channel tabs (Wholesale / DTC / TOTAL), SKU × month rows")
print(f"  ✅ Building Blocks tab: Raw 2025 | Seasonal Index | Deseasonalized | Base | Trend | Forecast")
print(f"  ✅ Inactivity rule: SKUs with 0 sales in last 6 months excluded from all forecasts")
print(f"  ✅ Per-channel growth rate overrides (CHANNEL_GROWTH_RATES dict)")



DEMAND PLANNING TOOL v3 CREATED SUCCESSFULLY!

Google Sheet Name: Demand_Planning_20260218_1622
URL: https://docs.google.com/spreadsheets/d/18330-VAQg6Cd-kWFujImQ8agIyUwqH_qWRzb6_obE6k

Sheets created:
  - Summary Dashboard
  - Product Type Breakdown
  - Direct-to-Consumer - Forecast
  - Wholesale - Forecast
  - TOTAL - Forecast
  - Agg by Month — Direct-to-Consumer
  - Agg by Month — Wholesale
  - Agg by Month — TOTAL
  - Building Blocks
  - QC Check

Total SKUs analyzed: 275
Channels: Direct-to-Consumer, Wholesale, TOTAL
Forecast period: Feb 2026 - Dec 2026

What's new in v3:
  ✅ Seasonality by PRODUCT TYPE (24-month pooled indices, not per-item)
  ✅ Seasonality table in Product Type Breakdown tab (month × product type)
  ✅ Forecast sheets show 2024 + 2025 actuals alongside 2026 forecast
  ✅ Actual vs Forecast columns color-coded (slate = actual, green = forecast)
  ✅ Channel total row added to each forecast sheet
  ✅ Aggregated By Month — 3 channel tabs (Wholesale / DTC / TOTAL), S